In [2]:
# Install necessary libraries
!pip install transformers flask flask-ngrok
!pip install tensorflow

# Import necessary packages
import os
import numpy as np
import pandas as pd
import torch
import tensorflow as tf
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Embedding
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import re


In [3]:
# Load GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# Define parameters for the RNN model
VOCAB_SIZE = 5000
EMBEDDING_DIM = 128
RNN_UNITS = 256
BATCH_SIZE = 64

# Build RNN Model
rnn_model = tf.keras.Sequential([
    Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_shape=(None,)),
    LSTM(RNN_UNITS, return_sequences=True, stateful=False, recurrent_initializer='glorot_uniform'),
    Dense(VOCAB_SIZE)
])

# Summary of the RNN model
rnn_model.summary()


In [5]:
# Sample dataset
data = [
    {"query": "What is your return policy?", "response": "Our return policy lasts 30 days..."},
    {"query": "How can I track my order?", "response": "You can track your order through..."},
    {"query": "What payment methods are accepted?", "response": "We accept Visa, MasterCard..."}
]

# Convert dataset to DataFrame
df = pd.DataFrame(data)

# Tokenize text for RNN model
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", '', text)
    return text

df['query'] = df['query'].apply(preprocess_text)


In [6]:
# Tokenizer for RNN input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize and pad the queries
tokenizer_rnn = Tokenizer(num_words=VOCAB_SIZE)
tokenizer_rnn.fit_on_texts(df['query'])
sequences = tokenizer_rnn.texts_to_sequences(df['query'])
input_data = pad_sequences(sequences, padding='post')

# Train RNN model
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
rnn_model.fit(input_data, input_data, epochs=100, batch_size=BATCH_SIZE)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - loss: 14.1600
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 11.1950
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 9.0640
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 8.2115
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 8.1108
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 8.0140
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 7.8931
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 7.7759
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 7.6448
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 7.5209
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 7.3635
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 7.1308
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 6.7144
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 6.1700
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 5.9255
Epoch 16/100
1/1 ━━━━

In [7]:
def generate_gpt_response(context):
    input_ids = tokenizer.encode(context, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Test GPT response generation
test_query = "How can I track my order?"
context = preprocess_text(test_query)
response = generate_gpt_response(context)
print(f"Response: {response}")


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Response: how can i track my order?

I have a question about the shipping. I have a question about the shipping.

I have a question about the shipping.

I have a question about the shipping.

I have a question about the shipping.

I have a question about the shipping.

I have a question about the shipping.

I have a question about the shipping.

I have a question about the shipping.

I have a


In [8]:
# Function to maintain conversation context and generate a response
def get_response_rnn_gpt(query, history=[]):
    # Update context with RNN
    processed_query = preprocess_text(query)
    history.append(processed_query)

    # Convert history to text
    context_text = ' '.join(history)

    # Generate response using GPT
    response = generate_gpt_response(context_text)
    return response

# Test combined RNN and GPT response
history = []
user_query = "What payment methods are accepted?"
response = get_response_rnn_gpt(user_query, history)
print(f"Response: {response}")


Response: what payment methods are accepted by the customer.

The customer must provide the following information:

The name and address of the customer.

The name and address of the customer's bank account.

The name and address of the customer's credit card.

The name and address of the customer's credit card number.

The name and address of the customer's credit card number.

The name and address of the customer's credit card number.

The
